In [7]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [8]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [9]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [10]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [11]:
# data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
# print(data_pk2_path)
print(media_pk2_path)

/Users/hodung/Desktop/Workspace/silkroad/pk2-extractor/LoadTextData/data/Media.pk2


In [12]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [13]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [14]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [15]:
_media_file_stream = open(media_pk2_path, 'rb')

In [16]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [17]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [18]:
blowfish_checksum_decoded = "Joymax Pak File"

In [19]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [20]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [21]:
Root = read_blocks_at(256)

In [22]:
blocks_in_memory = {
  "": Root
}

In [23]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [24]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  

def convert_string(value):
    try:
        # First, try to convert to an integer
        return int(value)
    except ValueError:
        try:
            # If it fails, try to convert to a float
            return float(value)
        except ValueError:
            # If both conversions fail, return the original string
            return value


In [25]:
text_data_directory = os.path.join('server_dep', 'silkroad', 'textdata')
text_data_directory

'server_dep/silkroad/textdata'

## Load Text Data

### TextUISystem.txt

In [26]:
text_ui_system_list_file = os.path.join(text_data_directory, 'TextUISystem.txt')
text_ui_system_list_file

'server_dep/silkroad/textdata/TextUISystem.txt'

In [27]:
text_ui_system_files = get_lines(text_ui_system_list_file)
text_ui_system_files

['TextUISystem_100.txt',
 'TextUISystem_1000.txt',
 'TextUISystem_1100.txt',
 'TextUISystem_1200.txt',
 'TextUISystem_1300.txt',
 'TextUISystem_1400.txt',
 'TextUISystem_1500.txt',
 'TextUISystem_1600.txt',
 'TextUISystem_1700.txt',
 'TextUISystem_1800.txt',
 'TextUISystem_1900.txt',
 'TextUISystem_200.txt',
 'TextUISystem_2000.txt',
 'TextUISystem_2100.txt',
 'TextUISystem_2200.txt',
 'TextUISystem_2300.txt',
 'TextUISystem_2400.txt',
 'TextUISystem_2500.txt',
 'TextUISystem_2600.txt',
 'TextUISystem_2700.txt',
 'TextUISystem_2800.txt',
 'TextUISystem_2900.txt',
 'TextUISystem_300.txt',
 'TextUISystem_3000.txt',
 'TextUISystem_3100.txt',
 'TextUISystem_3200.txt',
 'TextUISystem_3300.txt',
 'TextUISystem_3400.txt',
 'TextUISystem_3500.txt',
 'TextUISystem_3600.txt',
 'TextUISystem_3700.txt',
 'TextUISystem_3800.txt',
 'TextUISystem_3900.txt',
 'TextUISystem_400.txt',
 'TextUISystem_4000.txt',
 'TextUISystem_4100.txt',
 'TextUISystem_4200.txt',
 'TextUISystem_4300.txt',
 'TextUISystem_4

In [28]:
df_text_ui_system = pd.DataFrame(columns=range(17))

for text_ui_system_file in text_ui_system_files:
  file_name = os.path.join(text_data_directory, text_ui_system_file)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 17:
      continue
    df_text_ui_system.loc[len(df_text_ui_system)] = values
    


In [29]:
df_text_ui_system

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,1,UIC_STT_FONTNAME,굴림체,0,0,0,0,0,Arial,Tahoma,0,0,0,0,0,0
1,1,2,UIIS_STT_MSG_INSTALL_START,SilkRoad Online' 의 설치를 시작합니다.,0,0,0,0,0,Install 'Silkroad'.,"Cài đặt ""Con Đường Tơ Lụa""",0,0,0,0,0,0
2,1,3,UIIS_STT_MSG_INSTALL_FOLDER,SilkRoad Online' 을 다음 폴더에 설치합니다.,0,0,0,0,0,Install 'Silkroad' to the following folder.,"Cài đặt ""Con Đường Tơ Lụa"" vào thư mục sau",0,0,0,0,0,0
3,1,4,UIIS_STT_MSG_INSTALL_END,SilkRoad Online' 의 설치가 완료 되었습니다,0,0,0,0,0,Silkroad' has been successfully installed.,Con Đường Tơ Lụa đã được cài đặt thành công,0,0,0,0,0,0
4,1,5,UIIS_STT_INSTALL_FOLDER,설치 폴더,0,0,0,0,0,Installation Folder,Thư mục cài đặt,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6388,1,895,UIIT_STT_MOVEKEY,이동키,0,0,0,0,0,Move Key(keyboard button),Phím di chuyển,0,0,0,0,0,0
6389,1,896,UIIT_STT_INTERFACEKEY,인터페이스키,0,0,0,0,0,Interface,Phím giao diện,0,0,0,0,0,0
6390,1,897,UIIT_STT_SHORTENKEY,단축키,0,0,0,0,0,Shortcut,Phím tắt,0,0,0,0,0,0
6391,1,898,UIIT_STT_SHORTENKEY_USER_RULE,단축키 설정 사용자 지정,0,0,0,0,0,Custom shortcut,Thiết lập các phím chức năng,0,0,0,0,0,0
